In [4]:
import numpy as np
import os
import random
from astropy.io import fits

In [2]:
datadir = 'hw7_data_export'
fext = '.fits'

objfile = []
darkfile = []

all_files = os.listdir(datadir)
#print(all_files)

for f in all_files:
    
    if 'stars_13s_' in f:
        objfile.append( f[  : -5] ) 
    elif 'dark_13s_' in f:
        darkfile.append( f[  : -5] )

objfile

['stars_13s_7',
 'rdpharocor_stars_13s_1',
 'stars_13s_8',
 'stars_13s_4',
 'stars_13s_2',
 'stars_13s_5',
 'stars_13s_3',
 'stars_13s_6',
 'stars_13s_1']

In [3]:
objfile, darkfile = sorted(objfile), sorted(darkfile)

In [4]:
nobj = len(objfile)
ndark = len(darkfile)
print(nobj,ndark)

9 5


In [12]:
folder_path = "hw7_data_export"
fits_files = [f for f in os.listdir(folder_path) if f.endswith(".fits")]
if objfile:
    random_obj_filename = random.choice(objfile) + fext
    with fits.open(os.path.join(folder_path, random_obj_filename)) as hdul:
        data = hdul[0].data
        ny, nx = data.shape        

print(f"Data array dimensions: ny = {ny}, nx = {nx}")
print(f"Number of object files (nobj): {nobj}")
print(f"Number of dark files (ndark): {ndark}")

Data array dimensions: ny = 1024, nx = 1024
Number of object files (nobj): 9
Number of dark files (ndark): 5


In [13]:
objcube = np.zeros((nobj, ny, nx), dtype = np.float64)
darkcube = np.zeros((ndark, ny, nx), dtype = np.float64 )

In [19]:
for i, fname in enumerate(objfile):
    with fits.open(fname + fext) as hdul:
        objcube[i] = hdul[0].data
        if i == nobj - 1:
            objhead = hdul[0].header

for i, fname in enumerate(darkfile):
    with fits.open( fname + fext ) as hdul:
        darkcube[i] = hdul[0].data
        if i == ndark - 1:
            darkhead = hdul[0].header

print(f"Object data cube shape:  {objcube.shape}")
print(f"Object DATE-OBS:         {objhead.get('DATE-OBS')}")
print(f"Object TIME-OBS:         {objhead.get('TIME-OBS')}")

print(f"Dark data cube shape:    {darkcube.shape}")
print(f"Dark DATE-OBS:           {darkhead.get('DATE-OBS')}")
print(f"Dark TIME-OBS:           {darkhead.get('TIME-OBS')}")

Object data cube shape:  (9, 1024, 1024)
Object DATE-OBS:         2002-01-27
Object TIME-OBS:         
Dark data cube shape:    (5, 1024, 1024)
Dark DATE-OBS:           2002-01-27
Dark TIME-OBS:           12:43:54


### Below is Hw7_sana.py which contains the data cube of obj_sub_dark ###

In [5]:

# Taking the data from the folder
folder_path = "hw7_data_export"
fits_files = [f for f in os.listdir(folder_path) if f.endswith(".fits")]

dark_files = sorted([f for f in fits_files if f.startswith("dark_13s_")])
object_files = sorted([f for f in fits_files if f.startswith("stars_13s_")])

dark_stack = []
for file in dark_files:
    with fits.open(os.path.join(folder_path, file)) as hdul:
        dark_stack.append(hdul[0].data)
dark_stack = np.array(dark_stack)

# Median combine the dark frames
median_dark = np.median(dark_stack, axis=0)

print("\nMedian dark pixel [217, 184]:", median_dark[217, 184],'\n')

# Get header from the first dark frame and add HISTORY
with fits.open(os.path.join(folder_path, dark_files[0])) as hdul:
    header = hdul[0].header
header.add_history("A Median combination dark frame is created.")


# Save the median dark frame to a new FITS file
fits.writeto("dark_13s_med.fits", median_dark, header=header, overwrite=True)


# Subtract the median dark from each object frame
object_subtracted = []
for file in object_files:
    with fits.open(os.path.join(folder_path, file)) as hdul:
        obj_data = hdul[0].data
        subtracted = obj_data - median_dark
        object_subtracted.append(subtracted)

# Print pixel value [217, 184] before and after subtraction for the first object frame
with fits.open(os.path.join(folder_path, object_files[0])) as hdul:
    original_pixel = hdul[0].data[217, 184]
print("\nOriginal stars_13s_1.fits pixel [217, 184]:", original_pixel,'\n')
print("\nSubtracted stars_13s_1.fits pixel [217, 184]:", object_subtracted[0][217, 184],'\n')

# Save the first subtracted object frame to a new FITS file
fits.writeto("hw7_prob2_graph1_sana.fits", object_subtracted[0], overwrite=True)


Median dark pixel [217, 184]: 22.0 


Original stars_13s_1.fits pixel [217, 184]: 10879.0 


Subtracted stars_13s_1.fits pixel [217, 184]: 10857.0 



In [6]:
def normmedcomb(data_cube, norm_region=None):
    """
    Implements normalized median combination.
    
    Parameters:
    -----------
    data_cube : ndarray
        3D array (n_frames, y, x) of dark-subtracted sky flux.
    norm_region : tuple, optional
        ((y1, x1), (y2, x2)) defining normalization region.
        Default: entire image.
    
    Returns:
    --------
    combined_image : ndarray
        2D normalized, median-combined image.
    norm_factors : ndarray
        1D array of normalization factors for each frame.
    """
    data = np.copy(data_cube)
    n_frames, ny, nx = data.shape
    
    # Default normalization region is full image
    if norm_region is None:
        norm_region = ((0, 0), (ny, nx))
    
    (y1, x1), (y2, x2) = norm_region
    
    # Handle negative indices
    y2 = ny + y2 if y2 < 0 else y2
    x2 = nx + x2 if x2 < 0 else x2
    
    # Compute normalization factors
    norm_factors = []
    for i in range(n_frames):
        region = data[i, y1:y2, x1:x2]
        median_val = np.median(region)
        norm_factors.append(median_val)
        data[i] /= median_val  # Normalize frame
    
    norm_factors = np.array(norm_factors) # Store factors
    
    # Median combine
    combined_image = np.median(data, axis=0)
    
    return combined_image, norm_factors

In [7]:
# Given region
norm_region = ((225, 225), (-225, -225))
# Use function
combined_image, norm_factors = normmedcomb(object_subtracted, norm_region)

# Save to FITS
from astropy.io import fits
fits.writeto('sky_13s_mednorm.fits', combined_image, overwrite=True)

# Print normalization factors
print("Normalization factors:", norm_factors)    # To be included in the homework file

Normalization factors: [10864. 11057. 10536. 10559. 11046. 10623.  9916.  9916.]
